In [ ]:
import tensorflow as tf
import struct
import numpy as np

path = "../IAI_5/data/"

# Helper functions to load MNIST data from IDX files
def load_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        if magic != 2051:
            raise ValueError(f"Invalid magic number {magic} in file: {file_path}")
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        images = image_data.reshape((num_images, rows, cols, 1))  # Add channel dimension for CNN
        return images / 255.0  # Normalize pixel values to [0, 1]

def load_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num_labels = struct.unpack('>II', f.read(8))
        if magic != 2049:
            raise ValueError(f"Invalid magic number {magic} in file: {file_path}")
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

# Load the MNIST dataset
train_images = load_mnist_images(path + 'train-images.idx3-ubyte')
train_labels = load_mnist_labels(path + 'train-labels.idx1-ubyte')
test_images = load_mnist_images(path + 't10k-images.idx3-ubyte')
test_labels = load_mnist_labels(path + 't10k-labels.idx1-ubyte')

# Define the CNN model
model = tf.keras.Sequential([
    # Convolutional layer 1: 32 filters, 3x3 kernel, ReLU activation
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    # Max pooling layer 1: 2x2 pool size
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Convolutional layer 2: 64 filters, 3x3 kernel, ReLU activation
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # Max pooling layer 2: 2x2 pool size
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Flatten layer: Converts 2D feature maps to 1D feature vector
    tf.keras.layers.Flatten(),

    # Fully connected (Dense) layer: 128 neurons, ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),

    # Dropout layer: Drop 30% of neurons during training to reduce overfitting
    tf.keras.layers.Dropout(0.3),

    # Output layer: 10 neurons (one for each digit), softmax activation for probabilities
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")
